In [1]:
# Trabajo:
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error

#Lectura de los datos:
os.chdir("F:")
data = pd.read_csv("STA_ID_IALICANT2_STARTdate_20140101_ENDdate_20150225.csv")

#                                             Preparación de los datos

# Temperatura alta,baja y humedad media. Se utilizan 7 días previos:
Xin=data[['tempHigh','tempLow','humidityAvg']].values#,'pressureMax','precipRate','pressureMin','precipTotal','humidityHigh','humidityLow',]].values
Xin=Xin.astype('float32')
X_1=Xin[0:-9,:]
X_2=Xin[1:-8,:]
X_3=Xin[2:-7,:]
X_4=Xin[3:-6,:]
X_5=Xin[4:-5,:]
X_6=Xin[5:-4,:]
X_7=Xin[6:-3,:]

# Día:
epoch=data[['epoch']].values
epoch=epoch.astype('float32')
epoch /= max(epoch)
epoch -= (np.std(epoch))
epoch=epoch[7:-2]

# Tasa de precipitación:
p_rate=data[['precipRate']].values
p_rate=p_rate.astype('float32')
p_rate /= max(p_rate)
p_rate -= (np.std(p_rate))
p_rate=p_rate[6:-3]

# Presión máxima y mínima (se utilizan para hallar la diferencia de presión):
press=data[['pressureMax','pressureMin']].values
press=press.astype('float32')
inc_p=press[:,0]-press[:,1]
inc_p /= max(inc_p)
inc_p -= (np.std(inc_p))
X_i2=inc_p[6:-3]
pressure=epoch
for i in range(0, epoch.size-1) :
    pressure[i]=X_i2[i]

# Humedad máxima y mínima (se utilizan para hallar la diferencia de humedad):
hum=data[['humidityHigh','humidityLow']].values
hum=hum.astype('float32')
inc_h=hum[:,0]-hum[:,1]
inc_h /= max(inc_h)
inc_h -= (np.std(inc_h))
X_i2=inc_h[6:-3]
humidity=epoch
for i in range(0, epoch.size-1) :
    humidity[i]=X_i2[i]

# Se unen todos los vectores en uno solo:
# Si se quiere visualizar el tamaño: X.shape
X = np.hstack((X_1, X_2, X_3, X_4, X_5,X_6,X_7, epoch,pressure, humidity,p_rate))

# Predicción: lo que se quiere hacer a la red aprender, que en nuestro caso es la temperatura media
Y1=data['tempAvg'].values
Y1=Y1.astype('float32')


#                   Método final: SDGRegressor + Random Forest + SVM
# Método 4: SDGRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr1 = make_pipeline(StandardScaler(), SGDRegressor(max_iter=2000, tol=1e-3))
regr_1=regr1

# Método 6: Random Forest
from sklearn.ensemble import RandomForestRegressor
regr2 = RandomForestRegressor(n_estimators=500)
regr_2=regr2

# Método 1: SVM
from sklearn import svm
regr3 = svm.SVR()

# Unión de los tres métodos
from sklearn.ensemble import StackingRegressor
estimators = [('rf', regr_2),('sdg',regr_1)]
clf_1 = StackingRegressor(estimators=estimators, final_estimator=regr3)
clf_2=clf_1
clf_3=clf_1

In [2]:
def modelo_final(X_predecir, Y_predecir):
    result1=clf_1.predict(X_predecir)
    result2=clf_2.predict(X_predecir)
    result3=clf_3.predict(X_predecir)
    
    error=abs(Y_predecir.T[0]-result1)
    error2=np.where(error>=2)
    n_errores=error2[0].size
    tasa_errores1=n_errores/result3.size
    print('Día siguiente:')
    print('La tasa de error es de: ',("%.2f" % round(tasa_errores1*100, 2)), '%')
    print('El MSE obtenido es de: ',("%.4f" % round(mean_squared_error(Y_predecir.T[0], result1), 4)))

    error=abs(Y_predecir.T[1]-result2)
    error2=np.where(error>=2)
    n_errores=error2[0].size
    tasa_errores2=n_errores/result3.size
    print()
    print('Dos días después:')
    print('La tasa de error es de: ',("%.2f" % round(tasa_errores2*100, 2)), '%')
    print('El MSE obtenido es de: ',("%.4f" % round(mean_squared_error(Y_predecir.T[1], result2), 4)))
    
    error=abs(Y_predecir.T[2]-result3)
    error2=np.where(error>=2)
    n_errores=error2[0].size
    tasa_errores3=n_errores/result3.size
    print()
    print('Tres días después:')
    print('La tasa de error es de: ',("%.2f" % round(tasa_errores3*100, 2)), '%')
    print('El MSE obtenido es de: ',("%.4f" % round(mean_squared_error(Y_predecir.T[2], result3), 4)))
    
    print()
    print('Media:')
    print('La tasa de error es de: ',("%.2f" % round((tasa_errores1+tasa_errores2+tasa_errores3)/3*100,2)), '%')
    print('El MSE obtenido es de: ',("%.4f" % round((mean_squared_error(Y_predecir.T[0], result1)
                                                     +mean_squared_error(Y_predecir.T[1], result2)
                                                     +mean_squared_error(Y_predecir.T[2], result3))/3,4)))
    return

In [3]:
from sklearn.model_selection import train_test_split
i_train, i_test = train_test_split(np.arange(X.shape[0]), test_size=0.15)

# Predecir el día siguiente (mañana)
Y=Y1[7:-2]
X_test=X[i_test]
Y_test1=Y[i_test]
X_train=X[i_train]
Y_train=Y[i_train]

clf_1.fit(X_train, Y_train)

# Predecir el día posterior
Y=Y1[8:-1]
Y_test2=Y[i_test]
X_train=X[i_train]
Y_train=Y[i_train]

clf_2.fit(X_train, Y_train)

# Predecir tres días después
Y=Y1[9:]
Y_test3=Y[i_test]
X_train=X[i_train]
Y_train=Y[i_train]

clf_3.fit(X_train, Y_train)

StackingRegressor(estimators=[('rf', RandomForestRegressor(n_estimators=500)),
                              ('sdg',
                               Pipeline(steps=[('standardscaler',
                                                StandardScaler()),
                                               ('sgdregressor',
                                                SGDRegressor(max_iter=2000))]))],
                  final_estimator=SVR())

In [4]:
Y_test1=np.expand_dims(Y_test1, axis=0)
Y_test2=np.expand_dims(Y_test2, axis=0)
Y_test3=np.expand_dims(Y_test3, axis=0)

Y_test=np.hstack((Y_test1.T,Y_test2.T,Y_test3.T))

modelo_final(X_test, Y_test)

Día siguiente:
La tasa de error es de:  20.97 %
El MSE obtenido es de:  2.9802

Dos días después:
La tasa de error es de:  17.74 %
El MSE obtenido es de:  3.0190

Tres días después:
La tasa de error es de:  16.13 %
El MSE obtenido es de:  3.5891

Media:
La tasa de error es de:  18.28 %
El MSE obtenido es de:  3.1961
